## Imports

> [Link to Brookings](https://www.brookings.edu/multi-chapter-report/vital-statistics-on-congress/)

> [Link to BallotPedia](https://ballotpedia.org/Legislative_Branch)

> [Link to WikiPedia](https://en.wikipedia.org/wiki/List_of_United_States_Senate_elections)

> [Link to Wiki-Category (U.S. Senator)](https://commons.wikimedia.org/wiki/Category:Senators_of_the_United_States)

In [1]:
import functions as dlf
import pandas as pd
import numpy as np
import requests
import regex
import pickle
from importlib import reload
from bs4 import BeautifulSoup
from genderize import Genderize

In [ ]:
reload(dlf)

In [96]:
df = pd.read_csv('Senate_generals_thru_2016')
df_gend = df.copy()
# display(df.head(10))
display(df.tail())
# df.info()

,%,Turnout,Incumb_Y,State,Cln_name,Year,Terms_in_office,Party_enc,First_name,Seats_up%,Seats_before%
5583,1.01,320467.0,0,Vermont,Peter Diamondstone,2016,0,T,Peter,0.000000,0.00
5584,59.01,3243317.0,1,Washington,Patty Murray,2016,4,D,Patty,0.227273,0.44
5585,40.99,3243317.0,0,Washington,Chris Vance,2016,0,R,Chris,0.444444,0.54
5586,50.20,2947345.0,1,Wisconsin,Ron Johnson,2016,1,R,Ron,0.444444,0.54
5587,46.80,2947345.0,0,Wisconsin,Russ Feingold,2016,0,D,Russ,0.227273,0.44


In [97]:
def gender_guesser(inp_df, count_max=1000, g_thresh=0.6, online=False):
    import pandas as pd
    import numpy as np
    from genderize import Genderize
    
    ## Instantiate Genderize + making varibles for use
    g_izer = Genderize()
    out_df = inp_df.copy()
    count = 0
    
    ## Q.C. for formatting
    if 'Gender' not in out_df.columns:
        out_df['Gender'] = 'None'

    ## Looping through ea. name and making a guess
    for idx, name in zip(out_df.index, out_df['First_name']):
        ## Filtering out previously filled items
        if out_df['Gender'].loc[idx] == 'None':
            pred = None
            ## Try statement for api error results
            #try:
            ## Pull guess dict from api or manufacture one
            if online:
                guess = g_izer.get([name])[0]
            else:
                guess = {}
                guess['gender'] = np.random.choice(['male', 'female'])
                guess['probability'] = np.random.randint(0.01, high=1)

            ## Executing custom threshold for gender probability
            if guess['gender'] == 'male':
                if guess['probability'] < g_thresh:
                    pred = 'F'
                else:
                    pred = 'M'

            if guess['gender'] == 'female':
                if guess['probability'] < g_thresh:
                    pred = 'M'
                else:
                    pred = 'F'

            ## Setting the prediction into the correct index
            out_df.at[idx, 'Gender'] = pred
            count += 1
                
            #except:
            #    print('Pred Error!')
            #    #print(e)
            #    break
        
        ## Pseudo call limit for offline testing
        if (1 < count) & (count % count_max == 0):
            print(('***'*10), f'Hit {count_max} name limit.', ('***'*10))
            return out_df
        
    print(('***'*10),f'Finished {count} names. Remaining available: {count_max-count}.',('***'*10))
    return out_df

In [110]:
df_gend = gender_guesser(df_gend, online=True)

****************************** Finished 588 names. Remaining available: 412. ******************************


In [101]:
df_gend['Gender'].value_counts()

None    3588
M       1844
F         94
Name: Gender, dtype: int64

In [104]:
df_gend['Gender'].value_counts()

M       2757
None    2588
F        150
Name: Gender, dtype: int64

In [106]:
df_gend['Gender'].value_counts()

M       3616
None    1588
F        259
Name: Gender, dtype: int64

In [109]:
df_gend['Gender'].value_counts()

M       4448
None     588
F        406
Name: Gender, dtype: int64

In [151]:
holder_m = ['James Eli Watson', 'Selden Spencer', 'Forsman', 'Trudell', 
          'Robert La Follette', 'Cherry', 'France', 'Eickelberg', 
          'Day', 'Kindler', 'Carey', 'Duly McCone', 'Huey Long', 'Lanier', 
          'Glen Corkery', 'Starr', 'Hager', 'Verhei', 'Happy Chandler', 
          'Philo Farnsworth', 'Ona Searles', 'Lewis Haley Tiley', 'Cecil Creel',
          'John Marion Morris', 'Lynn Frazier', 'Glen Taylor', 'Hanna',
          'Lynn Stumbaugh', 'Alexis DuPont Bayard', 'Arthur Vivian Watkins',
          'Val Bjornson', 'Laurence Pickett', 'Lou Welch', 'John Little McClellan',
          'Cecil Underwood', 'Melvin Babcock Morgan', 'Cecil Heftel', 'Cecil Pitard',
          'Laurence Burton', 'Lynn Henderson', 'Lorin Oxley', 'Chic Hecht',
          'Arlyn Wollenburg', 'Lou Gold', 'Mel Logan', 'Lindsey Graham',
          'Lamar Alexander', 'John Neely Kennedy', 'Maclyn Stringer', 'Cam Cavasso',
          'LeAlan Jones', 'Rand Paul', 'Tracy Potter', 'David Lory VanDerBeek',
          'Paul Lindsey Sadler']

holder_n = ['Miscellaneous', 'Others', 'Write-In Candidates']

holder_c = ['Lawrence Gilfedder,Candidate for Governor in 1948. Ran for Governor in 1952 and 1954. Ran for Senate in 1958, 1960, 1962, 1964, 1966, and 1970.',
            'Jerry Zeldin, Laurel Springs', 'Howard Cannon, City Attorney of Las Vegas',
            'Douglas McKay,former Governor of Oregon  and United States Secretary of the Interior',
            'Efia Nwangaza*']

for i, name in zip(test.index, test['Cln_name']):
    for target in holder_m:
        if name == target:
            test.at[i, 'Gender'] = 'M'
            
    for target in holder_n:
        if name == target:
            test.at[i, 'Gender'] = 'None'
            
    if name == holder_c[0]:
        n_name = 'Lawrence Gilfedder'
        test.at[i, 'Cln_name'] = n_name
        test.at[i, 'Gender'] = 'M'
    elif name == holder_c[1]:
        n_name = 'Jerry Zeldin'
        test.at[i, 'Cln_name'] = n_name
        test.at[i, 'Gender'] = 'M'
    elif name == holder_c[2]:
        n_name = 'Howard Cannon'
        test.at[i, 'Cln_name'] = n_name
        test.at[i, 'Gender'] = 'M'
    elif name == holder_c[3]:
        n_name = 'Douglas Mckay'
        test.at[i, 'Cln_name'] = n_name
        test.at[i, 'Gender'] = 'M'
    elif name == holder_c[4]:
        n_name = 'Efia Nwangaza'
        test.at[i, 'Cln_name'] = n_name
            

In [152]:
# test = df_gend[df_gend['Gender'] == 'F'].copy()
test = test[test['Gender'] == 'F'].copy()
print(len(test['Gender']), '\n')
for name in test['Cln_name']:
    print(name)


407 

Elvina Beals
Josephine Bennett
Culla Bayhinger
Anne Henrietta Martin
Ella Boole
Rose Schneiderman
Virginia Green
Rachel Robinson
Rachel Robinson
Alva Adams
Antoinette Konikow
Merle Birmingham
Jessie Hughan
Ella Sanford
Anita Whitney
Charlotte Bohlin
Anna Storck
Ruth Hanna McCormick
Thelma Parkinson
Alva Adams
Clara Harrigal
Elisabeth Gilman
Rebecca Grecht
Cornelia Johnson
Fern Dobbins
Laetitia Conrad
Gertrude Wilson
Alva Adams
Elisabeth Gilman
Gladys Pyle
Anita Whitney
Mary Ellen Dooner
Alice Burke
Alice Burke
Josephine Sulston
Grace Carlson
Alice Burke
Georgia Cozzini
Genora Dollinger
Rubye Smith
Lillian Narins
Vivien Kellems
Thelma Ingersoll
Genora Dollinger
Genora Dollinger
Corliss Lamont
Ira Sassaman
Anna Chester
Isobel Cerney
Thelma Ingersoll
Rita Shaw
Suzanne Stevenson
Vivien Kellems
Clair Engle
Vivien Kellems
Evelyn Sell
Hazel Palmer
Corliss Lamont
Ethel Peterson
Louise Wensel
Georgia Cozzini
Lucia Cormier
Winifred Perry
Gladys Grauer
Maurine Neuberger
Maurine Neuberger
Ch

In [154]:
test[test['Turnout'].isna()]

,%,Turnout,Incumb_Y,State,Cln_name,Year,Terms_in_office,Party_enc,First_name,Seats_up%,Seats_before%,Gender
4406,100.0,NaN,0,Arkansas,Ophie Beltran,1998,0,T,Ophie,0.0,0.0,F


In [ ]:
## Make gender collection function #####################
## pickle/to_csv funct/df to Yish for help #############
## Keep year? Set_index as year (as datetime)
## Split % off as target
## OHE States, Party_enc
## RF/Tree Regressors 1st XGBoost?
## add first_name columns into df_fin ##################
## Make sure Turnout is ok as int. #####################

In [95]:
# test_df = df[df['Year'] == 2006].copy()
# display(test_df.head())
# display(test_df.info())

In [94]:
# test_df = gender_guesser(test_df, count_max=50)
# display(test_df.head())
# display(test_df.tail())